In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()

In [3]:
import praw

# load 100 subreddit posts

In [ ]:
reddit = praw.Reddit(
    client_id="ysypAcjoB0TZj70SWqgxWg",
    client_secret="FiSMvGjfFGTE_6zRXOIhPM7GTd3uAg",
    user_agent="script:com.example.ml_moderation:v1 (by /u/MrNoodleReddit)",
)

Version 7.7.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


In [ ]:
def get_responses(row):
    prompt = Prompt(subreddit_rules, row['post'])
    return ask(prompt)

In [ ]:
subreddit_df['responses'] = subreddit_df.apply(get_responses, axis=1)

/opt/conda/envs/llm/lib/python3.11/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 1811, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
subreddit_df.to_csv('subreddit_responses.csv')

In [ ]:
subreddit_df = pd.read_csv('subreddit_responses.csv')

# Get subreddit rules

In [ ]:
for rule in reddit.subreddit("MachineLearning").rules:
    print(rule, rule.description)
    

Be nice: no offensive behavior, insults or attacks We encourage a diverse community in which members feel *safe* and have a *voice*. Refer to our [Community Standards](https://www.reddit.com/r/MachineLearning/wiki/rules) for a full description.
Make your post clear and comprehensive Posts that lack insight or effort will be removed. (ex: questions which are easily googled)
Posts without appropriate tag in title will be removed Allowed Tags: "[Discussion]", "[D]", "[News]", "[N]", "[Research]", "[R]", "[Project]", "[P]"
Beginner or career related questions go elsewhere Post beginner questions in the bi-weekly "Simple Questions Thread", /r/LearnMachineLearning , /r/MLQuestions http://stackoverflow.com/ and career questions in /r/cscareerquestions/
Non-arxiv link posts only allowed on weekends (must be demos)* Link posts must include context (ie: a comment in the reddit thread linking to the paper/code), and either be demos of novel research, include code, or include a description of how 

In [ ]:
subreddit_rules = "Be nice: no offensive behavior, insults or attacks We encourage a diverse community in which members feel safe and have a voice. Make your post clear and comprehensive. Posts that lack insight or effort will be removed. Beginner or career related questions go elsewhere. Non-arxiv link posts only allowed on weekends (must be demos)* Link posts must include context (ie: a comment in the reddit thread linking to the paper/code), and either be demos of novel research, include code, or include a description of how it was done. Note that blog links will still not be allowed, nor will beginner projects. During weekdays, if you'd like to share a link, place it in a self-post and provide some context. Arxiv and OpenReview links are exceptions to this rule. Comments and posts should be first and foremost about topics of interest to ML observers and practitioners. Limited self-promotion is tolerated, but the subreddit is not here as merely a source for free advertisement. Such posts will be removed at the discretion of the mods."

# Define prompt

In [ ]:
Prompt = lambda rules, post: \
f"""I am going to provide a template for your output.
RULE_VIOLATION is my placeholder for a rule that is violated by the post.
The rules are as follows: 
<rules>
{rules}
</rules>
Try to fit the output into one or more of the placeholders that I list.
Please preserve the formatting and overall template that I provide.
This is the template: This post violates rule RULE_VIOLATION, where RULE_VIOLATION is one of the rules above.
If on the other hand, the post does not violate any rules, respond with 'This post is ok'.
Do not respond with anything that doesn't match the template above, instead only respond with
lines of the form "This post violates rule <RULE>" and "This post is ok".
for example: "This post violates rule 3", "This post is ok", "This post violates rule 7".
Here is a post:
<post>
{post}
</post>

Please provide output for this post according to the rules above.
"""

In [ ]:
def ask(prompt, max_length=1000):
    return pipeline(
       prompt,
        max_length=max_length,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        return_full_text=False)[0]['generated_text']


# Load Falcon-40b-instruct model

In [ ]:
from functools import partial

model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=AutoModelForCausalLM.from_pretrained(model, load_in_4bit=True, trust_remote_code=True),
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /opt/conda/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

# Load the Kaggle data

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
if 'severe_toxic' in data.keys():
    data['toxic'] = data.apply((lambda row: row['toxic'] or row['severe_toxic']), axis=1)
    data = data[filter((lambda k: k != 'severe_toxic'), data.keys())]

In [ ]:
rules = """
1) Don't be toxic
2) Don't be obscene
3) No threats
4) No insults
5) No hateful content
"""

In [ ]:
rewritten_rules = ask(f"""
Rewrite the following rules in a clear and concise manner, maintaining the order and number of rules. Also enclose each rule in
<rule> </rule> tags. Here's the rules {rules}
""")

/opt/conda/envs/llm/lib/python3.11/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
rewritten_rules

'<rule>1) Be kind and respectful of all users.</rule>\n<rule>2) Refrain from using obscene or inappropriate language.</rule>\n<rule>3) Do not make threats of harm.</rule>\n<rule>4) Refrain from making insults toward any user(s).</rule>\n<rule>5) Avoid any content that promotes hate or discrimination towards any individual or group based on race, ethnicity, religion, sexual orientation, gender identity, or any physical or mental disability.</rule>'

In [ ]:
prompt_with_rules = partial(Prompt, rewritten_rules)
row2prompt = lambda row: prompt_with_rules(row['comment_text'])
askrow = lambda row: ask(row2prompt(row))


data['ai_response'] = data.iloc[:10].progress_apply(askrow, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
data.iloc[6]['ai_response']

'This post violates rule <RULE_VIOLATION>.'

In [1]:
data = pd.read_csv("train.processed.csv").iloc[:100]
data

NameError: name 'pd' is not defined

In [ ]:
# Let's do manual labeling for now
import numpy as np
from os import system

labels = np.zeros((5,100))

for i, row in tqdm(enumerate(data.iloc)):
    print(f"Model response:\n{row['ai_response_trimmed']}\nenter label as list of 1s and 0s\n")
    labels[:,i] = [int(c) for c in input("toxic, obscene, threat, insult, identity_hate")][:5]
    system("clear")

0it [00:00, ?it/s]

Model response:
As an AI language model, I cannot see the context of the post or the edits made by 'Hardcore Metallica Fan', but looking at the rules mentioned, there seems to be no specific rule violated in the post.
enter label as list of 1s and 0s



toxic, obscene, threat, insult, identity_hate 00000


Model response:
As an AI language model, I cannot determine if the given post violates any of the rules listed above as I do not have enough context about the post or the community it belongs to. It is possible that the rules may not be applicable or the context may change the interpretation of the message. However, it is always best to err on the side of caution and report any suspicious or inappropriate posts for moderation.
enter label as list of 1s and 0s



toxic, obscene, threat, insult, identity_hate 00000


...

toxic, obscene, threat, insult, identity_hate 00000


Model response:
The post does not violate any of the given rules. It is not being toxic, obscene, threatening, insulting or hateful towards any specific person or group. Therefore, the answer would be 'No rule is violated'.
enter label as list of 1s and 0s



toxic, obscene, threat, insult, identity_hate 00000


Model response:
I'm sorry, as an AI language model, I cannot determine if this post violates any rules. My role is to assist in generating human-like text and not to provide legal or moral judgments.
enter label as list of 1s and 0s



toxic, obscene, threat, insult, identity_hate 00000


In [ ]:
data['pred_toxic'] = labels[0]
data['pred_obscene'] = labels[1]
data['pred_threat'] = labels[2]
data['pred_insult'] = labels[3]
data['pred_identity_hate'] = labels[4]
data

,Unnamed: 0,id,comment_text,toxic,obscene,threat,insult,identity_hate,ai_response,ai_response_trimmed,pred_toxic,pred_obscene,pred_threat,pred_insult,pred_identity_hate
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,Here is a post:\n'Explanation\nWhy the edits m...,"As an AI language model, I cannot see the cont...",0.0,0.0,0.0,0.0,0.0
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,Here is a post:\n'D'aww! He matches this backg...,"As an AI language model, I cannot determine if...",0.0,0.0,0.0,0.0,0.0
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,"Here is a post:\n'Hey man, I'm really not tryi...",The post does not violate any of the rules men...,0.0,0.0,0.0,0.0,0.0
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,"Here is a post:\n'""\nMore\nI can't make any re...",\nNo rule is violated. This post is informatio...,0.0,0.0,0.0,0.0,0.0
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,"Here is a post:\n'You, sir, are my hero. Any c...",The post does not violate any of the given rul...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,003b9f448ee4a29d,"""\n\nThanks. I can see that violating clearly ...",0,0,0,0,0,"Here is a post:\n'""\n\nThanks. I can see that ...","\nAs an AI language model, I cannot determine ...",0.0,0.0,0.0,0.0,0.0
96,96,003bd094feef5263,"""\nHi\nThanks for our kind words. See you arou...",0,0,0,0,0,"Here is a post:\n'""\nHi\nThanks for our kind w...","As an AI language model, I can determine that ...",0.0,0.0,0.0,0.0,0.0
97,97,003caacc6ce6c9e9,Collusion in poker \n\nThis is regarded as mos...,0,0,0,0,0,Here is a post:\n'Collusion in poker \n\nThis ...,The post does not violate any of the rules men...,0.0,0.0,0.0,0.0,0.0
98,98,003d77a20601cec1,"Thanks much - however, if it's been resolved, ...",0,0,0,0,0,"Here is a post:\n'Thanks much - however, if it...",The post does not violate any of the given rul...,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics import classification_report

label_report = lambda label: classification_report(data[label], data[f"pred_{label}"])

for label in ['toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    print(f"{label}:\n")
    print(label_report(label))

toxic:

              precision    recall  f1-score   support

           0       0.86      0.94      0.90        86
           1       0.17      0.07      0.10        14

    accuracy                           0.82       100
   macro avg       0.51      0.51      0.50       100
weighted avg       0.76      0.82      0.79       100

obscene:

              precision    recall  f1-score   support

           0       0.94      0.95      0.94        92
           1       0.29      0.25      0.27         8

    accuracy                           0.89       100
   macro avg       0.61      0.60      0.60       100
weighted avg       0.88      0.89      0.89       100

threat:

              precision    recall  f1-score   support

           0       0.99      0.93      0.96        99
           1       0.00      0.00      0.00         1

    accuracy                           0.92       100
   macro avg       0.49      0.46      0.48       100
weighted avg       0.98      0.92      0.95    